#First lab directions

## Sign up for Gurobi (If you already have a Gurobi account with your `cornell.edu` email, skip this step)
- Go to [Gurobi's website](https://portal.gurobi.com/iam/register/).
- Follow the sign-in instructions **with your Cornell email**.

## Request a Gurobi WLS license
- NOTE: You must be connected to Eduroam to complete this step.
- Go to the Gurobi [user portal](https://portal.gurobi.com/iam/licenses/request?type=academic).
- Make sure that ACADEMIC is selected (1).
- Under WLS Academic, click "GENERATE NOW!" (2).
![(1) is ACADEMIC, (2) is WLS Academic GENERATE NOW!](https://drive.google.com/uc?export=view&id=1lxTQHHHx61Yuz86ADgXvs8zRngsDm4dx)
- Agree to the terms and continue until it says the license was created.
- Click "OPEN WLS MANAGER" (3).
![License created textbox. OPEN WLS MANAGER is (3)](https://drive.google.com/uc?export=view&id=13YIabttApQA_mIz2y3c65o3XmNcL1kev)
- In the WLS manager, click the DOWNLOAD button under your license.
![Textbox with the license information. DOWNLOAD is (4)](https://drive.google.com/uc?export=view&id=168Qm-j4bCW35OhLkIKhYQD6SWGpnEix5)
- Enter a name for your key and select CREATE.
![ORIE3310 entered for the box Enter Application Name and (5) is CREATE](https://drive.google.com/uc?export=view&id=1BGQNMvOinu3YJftUYUwXRneZy2cDZmm0)
- In the following popup, select DOWNLOAD. This will download a `gurobi.lic` file to your computer. **Be sure to keep the file somewhere that you can find it again.**
![Textbox with (6) as DOWNLOAD](https://drive.google.com/uc?export=view&id=1BBfFyaXlgjUs5_XUhI776mIS6FFK13D_)

## Add your license to secrets
1. On the left bar of Colab click the key icon (7).
![The key icon is (7)](https://drive.google.com/uc?export=view&id=1LbCJYECXxO_6G0_FSrUfA_xuLpO_j2jZ)
2. Click "+ Add new secret" (8).
![The Add new secrete button is (8)](https://drive.google.com/uc?export=view&id=1OJhvkO86LAYDiG-0FD_j8UXhuZYcUUB-)
3. Open your `gurobi.lic` file on your computer with a text editor. You should see something of the form:
```# Gurobi WLS license file
# Your credentials are private and should not be shared or copied to public repositories.
# Visit https://license.gurobi.com/manager/doc/overview for more information.
WLSACCESSID=a2b3c4d5-6e7f-8g9h-i0j1-2k3l4m5n6o7p8
WLSSECRET=f1e2d3c4-5b6a-7890-1234-56789abcdef
LICENSEID=7654321
```
4. In the secrets prompt, put `WLSACCESSID` in the "Name" field and your key (the thing after the equal sign, so `a2b3c4d5-6e7f-8g9h-i0j1-2k3l4m5n6o7p8` in the example above) in the "Value" field.
![WLSACCESSID in the Name field and a2b3c4d5-6e7f-8g9h-i0j1-2k3l4m5n6o7p8 in the Value field](https://drive.google.com/uc?export=view&id=1S3sI-AKbuef4imBJfa27Q7I_-Cx6xhBg)
5. Repeat steps 2-4 of this part with the `WLSSECRET` and `LICENSEID` fields.
6. Check all the ticks for "Notebook access." **You will need to reopen the key menu and recheck these ticks for EVERY lab in this course**

# Lab

In [ ]:
!pip install pulp
!pip install gurobipy

In [ ]:
from pulp import *
from google.colab import userdata

Make sure that you have opened the Key menu, ticked Notebook access for each secret, and closed the key menu before running the next cell.

In [ ]:
options = {
"WLSACCESSID": userdata.get("WLSACCESSID"),
"WLSSECRET": userdata.get("WLSSECRET"),
"LICENSEID": int(userdata.get("LICENSEID")),
}

## Scheduling Final Exams from Enrollment Data
<br>
In this notebook, we will backwards through the modeling process. You will start by being presented with a data file, and a PuLP model file that uses that data to solve an optimization problem, and we will first try to understand the mathematics of what the code is doing, and finally, we will explore why that optimization model is producing useful information for the process of trying to decide on a final exam schedule. We will be introducing PuLP for python, which is an open source software package and this will call an integer programming solver called Gurobi, which is a state-of-the-art linear programming, and integer linear programming solver today, as well as throughout the semester to solve various optimization problems.
<br> <br>
For this problem, we are given the set of courses for the Cornell Fall 2024 semester that had final exams during exam period, and for each pair of these courses, we are given the number of students who are enrolled in both of them. That is, we are given:

- A set $F = \{ 1,\dots,m \}$, where each course $i
\in F$ has a final exam to be scheduled during finals period.

- The number $n_{ij}$ of students who are enrolled in both course $i$ and in course $j$, for each pair of distinct courses $i$ and $j$, $\{i,j\} \subseteq F$. (Quick check: do you know what the work "distinct" means in this context?)

In [ ]:
# imports
import pandas as pd

Make sure that you have placed the `coenrollmentdata.csv` file in the file explorer. You can do this by clicking the folder icon on the left and then dragging the file into the space that opens.

Run the cell bellow to read and print an example set of data that we will use to create and solve an input to our optimization model. The cell in row $i$ and column $j$ give the number of students enrolled in both course $i$ and course $j$, that is, the co-enrollment in those two courses. Note that the rows and columns are not indexed just as $\{1,2,\ldots,m\}$ but with a key associated the particular course. (For some of these, you might be able to guess where the value of the key comes from.) Since we are only interested in this data for pair of distinct courses, we will adopt the convention that the diagonal entries are all 0. What other properties does the data in this table satisfy?

In [ ]:
!if [ ! -f "coenrollmentdata.csv" ]; then wget https://github.com/engri-1101/orie-3310/raw/refs/heads/main/labs/lab1/coenrollmentdata.csv; fi

In [ ]:
example_data = pd.read_csv('coenrollmentdata.csv', index_col = 0)
display(example_data)

Let us now move on to creating and solving a model that uses this input data. We will create an integer linear program to capture a specific optimization model. We will go through the steps required to formulate and solve a linear program using Gurobi. <br> <br>

In total, we will go through the following steps:
 - Create the variables.
 - Define the constraints.
 - Define the objective function.
 - Calling the solver and printing the results.
<br>


Before that, run the following cell to import Gurobi and to declare it to be is the solver we will be using, and while there are other solvers available, we will not be going through them now.

In [ ]:
solver = GUROBI(manageEnv=True, envOptions=options)
model = LpProblem("FinalExamClique", LpMaximize)

##### Creating the Variables

Given the set $F$ of final exams, we will define integer variables $x_{i}, \forall i \in F $. These variables will be constrained to take on either the value 0 or the value 1. Run the cell bellow to create the set $F$ from the given input data. Rather than just letting $F$ be the integers $1,\ldots,m$, we will instead view the elements of $F$ as being the keys of the exams.

In [ ]:
F = list(example_data.index)
print(F)

Now we can create the integer variables $x_{i}$. The function [`LpVariable(name, cat=LpBinary)`](https://coin-or.github.io/pulp/technical/pulp.html#pulp.LpVariable) can be used to add variables to the model, where `name` is the name of the variable in the Gurobi model, and `cat=LpBinary` indicates that this variable can only take values of `0` or `1`.
In the cell below, loop over the set $F$ to add the variables $x_{i}$ to the model, storing them in dictionary `x`.

In [ ]:
x = {} # dictionary to store the variables x_ij

for i in F:
        x[i] = LpVariable('x'+str(i), cat=LpBinary)

# uncomment the cell below if you want to see a summary of the variables
# display(x)

##### Defining the constraints

We will have a constraint for each pair $i,j \in F$ for which the coenrollment value is positive. These constraints are added as follows using the [`+=`](https://coin-or.github.io/pulp/CaseStudies/a_blending_problem.html) operator:

In [ ]:
# adding the constraints
for i in F:
    for j in F:
        if i != j and example_data[i][j] == 0:
            model += (x[i] + x[j] <= 1)

#### Define the objective function

The final part before solving the model is adding the objective function, for which we use the [`+=`](https://coin-or.github.io/pulp/CaseStudies/a_blending_problem.html) operator again, this time with an expression instead of an inequality. This will maximize the objective as we declared when we created the model.


In [ ]:
model += lpSum(x[i] for i in F)

#### Calling the solver and printing the results

In [ ]:
print(model)

We are now ready to solve the model for the input data given. Running the cell bellow will call the [`solve(solver)`](https://coin-or.github.io/pulp/technical/pulp.html#pulp.LpProblem.solve) method to solve the program. The few lines of code after that will print the objective value, as will as the variables that have value greater than 0.

In [ ]:
model.solve(solver)
print('Objective value =',value(model.objective))
for var in model.variables():
    if var.varValue > 1e-10:
        print(var.name, round(value(var)))

solver.close()

We are done! We have only printed out the variables that have value greater than 0, since there are far fewer of them than are 0. In this case, we are doing things backwards, starting with the model and trying to understand why it is useful for decision-making in the context of final exams, but we will repeat this same process throughout the course, however, starting with the decision-making setting at hand, constructing an optimization model, and then coding it in PuLP. We will be coding larger models, so understanding the steps that we went throught to implement this integer program is important, as these are the basic steps one follows to code any model. <br> <br>



## Your turn

You will now apply your knowledge to create a new model. In this part, you will create a model that will schedule as many classes into one slot without conflict as possible.

We have defined the model for you.

In [ ]:
solver = GUROBI(manageEnv=True, envOptions=options)
model = LpProblem("FinalExamIndSet", LpMaximize)

### Define the variables

What decision variables do you need for this model? Define the decision variables below.

In [ ]:
x = {}

# TODO: Define the decision variables

### Define the constraints

We want to ensure that no two classes we schedule in the same slot have coenrollment. What constraint achieves this? Define the constraints below.

In [ ]:
# TODO: Define the constraints

### Define the objective

Define the objective function below.

In [ ]:
# TODO: Define the objective function

### Run the code below to solve the model!

For a sanity check, your model should find 152 courses that can be scheduled in the same slot.

In [ ]:
model.solve(solver)
print('Objective value =',value(model.objective))
to_schedule = []

for var in model.variables():
    if var.varValue > 1e-10:
        to_schedule.append(var.name[1:])

print('Classes to schedule: ', to_schedule)

solver.close()

### Moving forward

Now that you have a model to schedule classes into a single slot without conflicts, how could you use this model to schedule all classes into slots without conflicts?